# Cloud Level 2 Subsetter (L2SS) API 
This will demonstrate how to subset swath/L2 data with the data and services hosted on the cloud.


## Before you start (UAT login)
Before you begin this tutorial, make sure you have an account in the Earthdata Login UAT environment, which 
will be used for this notebook by visiting [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).
These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.

## Set Up Authentication

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [1]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


Now call the above function to set up Earthdata Login for subsequent requests

In [2]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

### Find a granule for subsetting

Below we call out a specific file/granule on which we will use the podaac L2 subsetter. Finding this information would complicate the tutorial, but there are several other demos that show how to use the search APIs.

##  Subset of a PO.DAAC Granule

We can now build onto the root URL in order to actually perform a transformation.  The first transformation is a subset of a selected granule.  _At this time, this requires discovering the granule id from CMR_.  That information can then be appended to the root URL and used to call Harmony with the help of the `request` library.

Above we show how to find a granule id for processing.

**Notes:**
  The L2 subsetter current streams the data back to the user, and does not stage data in S3 for redirects. This is functionality we will be adding over time.
  It doesn't work with URS backed files, which is coming in the next few weeks
  it only works on the show dataset, but 
    

In [3]:
bblat_max=1
bblat_min=0
bblon_max=1
bblon_min=0

In [4]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
bboxSubsetConfig = {
    'collection_id': 'C1234208437-POCLOUD',  #Jason-1 GDR SSHA version E NetCDF    
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1234220250-POCLOUD',      #JA1_GPR_2PeP001_092_20020118_182623_20020118_192236.nc
    'lat': '(' + str(bblat_min) + ':' + str(bblat_max) + ')',
    'lon': '(' + str(bblon_min) + ':' + str(bblon_max) + ')'
}

bbox_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?granuleid={granuleid}&subset=lat{lat}&subset=lon{lon}'.format(**bboxSubsetConfig)
print('Request URL', bbox_url)

Request URL https://harmony.uat.earthdata.nasa.gov/C1234208437-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?granuleid=G1234220250-POCLOUD&subset=lat(0:1)&subset=lon(0:1)


In [5]:
import shutil
with request.urlopen(bbox_url) as response, open('ogc_temp.nc', 'wb') as out_file:
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to ogc_temp.nc")

HTTPError: HTTP Error 504: Gateway Timeout

In [ ]:
import xarray as xa
ds = xa.open_dataset('ogc_temp.nc')
ds

In [ ]:
from pandas.plotting import register_matplotlib_converters

ds.ssha.plot()

## Verify the subsetting worked

Bounds are defined earlier 


In [ ]:
lat_max = ds.lat.max()
lat_min = ds.lat.min()

lon_min = ds.lon.min()
lon_max = ds.lon.max()


if lat_max < bblat_max and lat_min > bblat_min:
    print("Successful Latitude subsetting")
else:
    assert false

    
if lon_max < bblon_max and lon_min > bblon_min:
    print("Successful Longitude subsetting")
else:
    assert false
    

## Plot swath onto a map

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.scatter(ds.lon, ds.lat, lw=2, c=ds.ssha)
plt.colorbar()
plt.clim(-0.3, 0.3)

plt.show()